In [1]:
import pandas as pd

# Summarise RID detection limits

The RID report includes a table showing the proportion of values for each parameter that are at or below the detection limit (see e-mail from Eva received 23/10/2017 at 14.17 for details). This information can be derived from *concs_and_flows_rid_11-36_2016.csv*, which was generated in [this notebook](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_working_2016-17.ipynb#2.-Tabulate-raw-water-chemistry-and-flow).

In [41]:
# Read data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\Results\Loads_CSVs\concs_and_flows_rid_11-36_2016.csv')
df = pd.read_csv(in_csv)

# Remove unwanted cols
del df['station_id'], df['station_code'], df['station_name']
del df['rid_group'], df['ospar_region'], df['sample_date'], df['Qs_m3/s']

# Count non-null entries and transpose
df = (~df.isnull()).sum()

# Restructure
df = df.reset_index()
df.columns = ['name', 'count']
df['par'], df['unit'] = df['name'].str.split('_', 1).str
del df['name']

flg_df = df.query('unit == "flag"')
smp_df = df.query('unit != "flag"') 

df = pd.merge(smp_df, flg_df, how='left', 
              on='par', suffixes=['_smps', '_dls'])

df = df[['par', 'count_smps', 'count_dls']]
df.columns = ['par', 'n_samples', 'n_lod']

# Prop at LOD
df['lod_pct'] = 100*df['n_lod']/df['n_samples']

# Reorder to match Eva's table
par_list = ['pH', 'KOND', 'SPM', 'TOC', 'TOTP', 'PO4-P', 'TOTN', 'NO3-N', 'NH4-N', 
            'SiO2', 'Pb', 'Cd', 'Cu', 'Zn', 'As', 'Hg', 'Cr', 'Ni', 'Ag']
ord_df = pd.DataFrame({'par':par_list})

df = pd.merge(ord_df, df, how='left', on='par')

# Save
out_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Results\Loads_CSVs\lod_prop_rid_11-36_2016.csv')
df.to_csv(out_csv)

df

,par,n_samples,n_lod,lod_pct
0,pH,282,0,0.000000
1,KOND,283,2,0.706714
2,SPM,283,23,8.127208
3,TOC,283,0,0.000000
4,TOTP,283,0,0.000000
5,PO4-P,283,100,35.335689
6,TOTN,282,0,0.000000
7,NO3-N,282,3,1.063830
8,NH4-N,283,91,32.155477
9,SiO2,283,0,0.000000
